In [24]:
import pygame, sys
from button import Button
from classdef import Grid, Choice
from dijkstra import dijkstra
from showPath import showpath
import pygame
import logging
from gui import gui, notFound


In [25]:
def ASTAR_VIS():
    import pygame
    import math
    from queue import PriorityQueue

    WIDTH = 1280
    WIN = pygame.display.set_mode((WIDTH, WIDTH))
    pygame.display.set_caption("A* Path Finding Algorithm")
    MENU_TEXT = get_font(100).render("MAIN MENU", True, "#b68f40")
    MENU_RECT = MENU_TEXT.get_rect(center=(640, 100))
    SCREEN.blit(MENU_TEXT, MENU_RECT)

    RED = (255, 0, 0)
    GREEN = (0, 255, 0)
    BLUE = (0, 255, 0)
    YELLOW = (255, 255, 0)
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    PURPLE = (128, 0, 128)
    ORANGE = (255, 165 ,0)
    GREY = (128, 128, 128)
    TURQUOISE = (64, 224, 208)

    class Spot:
        def __init__(self, row, col, width, total_rows):
            self.row = row
            self.col = col
            self.x = row * width
            self.y = col * width
            self.color = WHITE
            self.neighbors = []
            self.width = width
            self.total_rows = total_rows

        def get_pos(self):
            return self.row, self.col

        def is_closed(self):
            return self.color == RED

        def is_open(self):
            return self.color == GREEN

        def is_barrier(self):
            return self.color == BLACK

        def is_start(self):
            return self.color == ORANGE

        def is_end(self):
            return self.color == TURQUOISE

        def reset(self):
            self.color = WHITE

        def make_start(self):
            self.color = ORANGE

        def make_closed(self):
            self.color = RED

        def make_open(self):
            self.color = GREEN

        def make_barrier(self):
            self.color = BLACK

        def make_end(self):
            self.color = TURQUOISE

        def make_path(self):
            self.color = PURPLE

        def draw(self, win):
            pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

        def update_neighbors(self, grid):
            self.neighbors = []
            if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): # DOWN
                self.neighbors.append(grid[self.row + 1][self.col])

            if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): # UP
                self.neighbors.append(grid[self.row - 1][self.col])

            if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): # RIGHT
                self.neighbors.append(grid[self.row][self.col + 1])

            if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): # LEFT
                self.neighbors.append(grid[self.row][self.col - 1])

        def __lt__(self, other):
            return False


    def h(p1, p2):
        x1, y1 = p1
        x2, y2 = p2
        return abs(x1 - x2) + abs(y1 - y2)


    def reconstruct_path(came_from, current, draw):
        while current in came_from:
            current = came_from[current]
            current.make_path()
            draw()


    def algorithm(draw, grid, start, end):
        count = 0
        open_set = PriorityQueue()
        open_set.put((0, count, start))
        came_from = {}
        g_score = {spot: float("inf") for row in grid for spot in row}
        g_score[start] = 0
        f_score = {spot: float("inf") for row in grid for spot in row}
        f_score[start] = h(start.get_pos(), end.get_pos())

        open_set_hash = {start}

        while not open_set.empty():
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()

            current = open_set.get()[2]
            open_set_hash.remove(current)

            if current == end:
                reconstruct_path(came_from, end, draw)
                end.make_end()
                return True

            for neighbor in current.neighbors:
                temp_g_score = g_score[current] + 1

                if temp_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = temp_g_score
                    f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                    if neighbor not in open_set_hash:
                        count += 1
                        open_set.put((f_score[neighbor], count, neighbor))
                        open_set_hash.add(neighbor)
                        neighbor.make_open()

            draw()

            if current != start:
                current.make_closed()

        return False


    def make_grid(rows, width):
        grid = []
        gap = width // rows
        for i in range(rows):
            grid.append([])
            for j in range(rows):
                spot = Spot(i, j, gap, rows)
                grid[i].append(spot)

        return grid


    def draw_grid(win, rows, width):
        gap = width // rows
        for i in range(rows):
            pygame.draw.line(win, GREY, (0, i * gap), (width, i * gap))
            for j in range(rows):
                pygame.draw.line(win, GREY, (j * gap, 0), (j * gap, width))


    def draw(win, grid, rows, width):
        win.fill(WHITE)

        for row in grid:
            for spot in row:
                spot.draw(win)

        draw_grid(win, rows, width)
        PLAY_MOUSE_POS = pygame.mouse.get_pos()
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
        QUIT_BUTTON = Button(image=None ,pos=(1230, 30), 
                        text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")

        OPTIONS_BACK = Button(image=None , pos=(60, 30), 
                        text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")
        for button in [QUIT_BUTTON, OPTIONS_BACK]:
            button.changeColor(OPTIONS_MOUSE_POS)
            button.update(SCREEN)
        MENU_TEXT_1 = get_font(1).render("first right click will set the start", True, "#b68f40")
        MENU_TEXT_2 = get_font(1).render("second click will set the goals of the maze ", True, "#b68f40")
        MENU_TEXT_3 = get_font(1).render("hold and drag the mouse to draw the walls of the maze ", True, "#b68f40")
        MENU_TEXT_4 = get_font(1).render("space to run the simulation , press {c} to restart", True, "#b68f40")
        MENU_RECT_1 = MENU_TEXT_1.get_rect(center=(500, 80))
        MENU_RECT_2 = MENU_TEXT_2.get_rect(center=(500, 180))
        MENU_RECT_3 = MENU_TEXT_3.get_rect(center=(500, 280))
        MENU_RECT_4 = MENU_TEXT_4.get_rect(center=(500, 380))            


        SCREEN.blit(MENU_TEXT_1, MENU_RECT_1)
        SCREEN.blit(MENU_TEXT_2, MENU_RECT_2)

        SCREEN.blit(MENU_TEXT_3, MENU_RECT_3)

        SCREEN.blit(MENU_TEXT_4, MENU_RECT_4)

        pygame.display.update()


    def get_clicked_pos(pos, rows, width):
        gap = width // rows
        y, x = pos

        row = y // gap
        col = x // gap

        return row, col


    def main(win, width):
        ROWS = 50
        grid = make_grid(ROWS, width)

        start = None
        end = None

        run = True
        while run:

            PLAY_MOUSE_POS = pygame.mouse.get_pos()
            OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
            QUIT_BUTTON = Button(image=None, pos=(1230, 30), 
                            text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")
        
            OPTIONS_BACK = Button(image=None, pos=(60, 30), 
                            text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")
            for button in [QUIT_BUTTON, OPTIONS_BACK]:
                button.changeColor(OPTIONS_MOUSE_POS)
                button.update(SCREEN)
            draw(win, grid, ROWS, width)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    run = False
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if OPTIONS_BACK.checkForInput(OPTIONS_MOUSE_POS):
                            play()
                    if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                            pygame.quit()
                            sys.exit()

                if pygame.mouse.get_pressed()[0]: # LEFT
                    pos = pygame.mouse.get_pos()
                    row, col = get_clicked_pos(pos, ROWS, width)
                    spot = grid[row][col]
                    if not start and spot != end:
                        start = spot
                        start.make_start()

                    elif not end and spot != start:
                        end = spot
                        end.make_end()

                    elif spot != end and spot != start:
                        spot.make_barrier()

                elif pygame.mouse.get_pressed()[2]: # RIGHT
                    pos = pygame.mouse.get_pos()
                    row, col = get_clicked_pos(pos, ROWS, width)
                    spot = grid[row][col]
                    spot.reset()
                    if spot == start:
                        start = None
                    elif spot == end:
                        end = None

                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE and start and end:
                        for row in grid:
                            for spot in row:
                                spot.update_neighbors(grid)

                        algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)
                   

                        

                    if event.key == pygame.K_c:
                        start = None
                        end = None
                        grid = make_grid(ROWS, width)

        pygame.quit()

    main(WIN, WIDTH)

In [26]:
def DIJKSTRA_VIS():
    WIDTH = 1280
    HEIGHT = 720

    # colors
    BLACK = (0, 0, 0)
    WHITE = (255, 255, 255)
    RED = (255, 0, 0)
    BG_COL = (0, 64, 128)


    # functions

    def redraw():
        win.fill(BLACK)  # fills the window after each frame
        pygame.draw.rect(win, BG_COL, (0, 0, round(WIDTH * 0.25), HEIGHT))  # draws frame for holding the options
        pygame.draw.rect(win, WHITE, mainGrid.getPos())  # draws frame for the node grid
        mainGrid.draw(win, buttonFont)
        pygame.draw.rect(win, BG_COL, (0, 0, round(WIDTH * 0.25), HEIGHT))  # draws frame for holding the options
        pygame.draw.rect(win, WHITE, mainGrid.getPos())  # draws frame for the node grid
        mainGrid.draw(win, buttonFont)
        PLAY_MOUSE_POS = pygame.mouse.get_pos()
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
        QUIT_BUTTON = Button(image=None, pos=(170, 650), 
                        text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")

        OPTIONS_BACK = Button(image=None, pos=(50, 650), 
                        text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")
        OPTIONS_BACK.changeColor(OPTIONS_MOUSE_POS)
        OPTIONS_BACK.update(SCREEN)
        
        QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        QUIT_BUTTON.update(SCREEN)


        pygame.display.update()


    # user input
    choice = Choice()
    #gui(choice)pygame.image.load("assets/Quit Rect.png")
    delay = 10
    #algorithms = [dijkstra]
    pathfinder = dijkstra
    # creating the grid
    mainGrid = Grid((WIDTH // 4), 0, WIDTH - (WIDTH // 4), HEIGHT, 10)

    # setting up a logger
    logging.basicConfig(filename='history.log', format='%(asctime)s %(message)s', level=logging.DEBUG)
    logger = logging.getLogger()  # getting an object called logger


    # pygaame window
    pygame.init()  # pygame window init
    win = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption('Pathfinder Visualization')
    clk = pygame.time.Clock()
    FPS = 60  # frames per second
    buttonFont = pygame.font.SysFont('Microsoft YaHei Light', 25)  # button font
    run = True  # for controlling the main loop

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # anyting between the lines are in the main loop
    while run:
        PLAY_MOUSE_POS = pygame.mouse.get_pos()
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
        QUIT_BUTTON = Button(image=None, pos=(170, 650), 
                        text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")

        OPTIONS_BACK = Button(image=None, pos=(50, 650), 
                        text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")
        for button in [QUIT_BUTTON, OPTIONS_BACK]:
            button.changeColor(OPTIONS_MOUSE_POS)
            button.update(SCREEN)
        clk.tick(40)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False  # stop the game if user tries to quit
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if OPTIONS_BACK.checkForInput(OPTIONS_MOUSE_POS):
                    play()
                if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    pygame.quit()
                    sys.exit()
            if pygame.mouse.get_pressed()[0]:  # only detects left click
                mainGrid.clickOnGrid(pygame.mouse.get_pos())  # checks if the click happened on grid
        if mainGrid.vizStarted:
            if mainGrid.start not in mainGrid.open and mainGrid.start not in mainGrid.closed:
                mainGrid.open.append(mainGrid.start)  # append the start node to the open list
                mainGrid.start.hcost = mainGrid.start.calcHcost(mainGrid.end)
                mainGrid.start.fcost = mainGrid.start.gcost + mainGrid.start.hcost
            if pathfinder(mainGrid, delay):
                if len(mainGrid.open) < 1:
                    run = False  # path not found
                else:
                    showpath(mainGrid.end)  # draws the path it has found
        redraw()
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    if len(mainGrid.open) < 1:  # no valid path found
        logger.info(f'Algorithm: {choice.algo} Speed: {choice.speed} Nodes in a row: {choice.nodes}. Path not found')
        notFound()
        DIJKSTRA_VIS()
        #
        #redraw()
        #print("novaildpass")
        #logger.info(f'Algorithm: {choice.algo} Speed: {choice.speed} Nodes in a row: {choice.nodes}. Path not found')
    else:
        logger.info(f'Algorithm: {choice.algo} Speed: {choice.speed} Nodes in a row: {choice.nodes}. Path found')
    pygame.quit()




In [27]:
pygame.init()

clock = pygame.time.Clock()

size = 600
menu_height = 80
num_rows = 20
line_width = size // num_rows
white = (255, 255, 255)
mat_white = (230, 230, 230)
black = (0, 0, 0)
green = (0, 255, 0)
mat_green = (46, 184, 46)
blue = (0, 0, 255)
mat_blue = (0, 102, 255)
child_node_blue = (70, 104, 163, 50)
path_cell_yellow = (203, 212, 40)
mat_red = (255, 80, 80)
bot = None
# destination = None
open_queue = []
close_queue = []
wall_cells = []
goal_cells = []
path = []
found_goal_cell = None
bot_init_cell = None
window = None 
play_area_rect = None
menu_area_rect = None
run_button = None
wall_button = None
bot_button = None
goal_button = None
clicked_button = None

def create_play_and_menu_area_sections():
    global play_area_rect, menu_area_rect
    play_area_rect = pygame.draw.rect(window, black, pygame.Rect(0, 0, size, size))
    menu_area_rect = pygame.draw.rect(window, black, pygame.Rect(0, size, size, menu_height))

def draw_menus():
    global run_button, wall_button, bot_button, goal_button
    # defining a font
    smallfont = pygame.font.SysFont('arial',20) # font face, and size
    OPTIONS_MOUSE_POS = pygame.mouse.get_pos()


    run_text = smallfont.render('RUN' , True , white) # Creating the text object
    wall_text = smallfont.render('WALL' , True , black) # Creating the text object
    bot_text = smallfont.render('BOT' , True , white) # Creating the text object
    goal_text = smallfont.render('GOAL' , True , white) # Creating the text object

    # run button
    run_button = pygame.draw.rect(window, mat_red, [10, size + menu_height // 3, 80, 30]) # a box around text to look like a button
    window.blit(run_text, (10+15+3, size + menu_height // 3 + 4))

    # wall button
    wall_button = pygame.draw.rect(window, mat_white, [110, size + menu_height // 3, 80, 30]) # a box around text to look like a button
    window.blit(wall_text, (110+15, size + menu_height // 3 + 4))

    # bot button
    bot_button = pygame.draw.rect(window, mat_blue, [210, size + menu_height // 3, 80, 30]) # a box around text to look like a button
    window.blit(bot_text, (210+15+3, size + menu_height // 3 + 4))

    # goal button
    goal_button = pygame.draw.rect(window, mat_green, [310, size + menu_height // 3, 80, 30]) # a box around text to look like a button
    window.blit(goal_text, (310+15-3, size + menu_height // 3 + 4))

    # pygame.draw.rect(window,color_dark,[100,100,80,30]) # a box around text to look like a button
    QUIT_BUTTON = Button(image=None, pos=(1047, 600), 
                        text_input="QUIT", font=get_font(75), base_color="White", hovering_color="Red")

    OPTIONS_BACK = Button(image=None, pos=(1047, 465), 
                        text_input="BACK", font=get_font(75), base_color="White", hovering_color="Green")
    OPTIONS_BACK.changeColor(OPTIONS_MOUSE_POS)
    OPTIONS_BACK.update(SCREEN)

    QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
    QUIT_BUTTON.update(SCREEN)
    pygame.display.update()
    

def main():
    global window, size, num_rows, bot, play_area_rect, menu_area_rect, clicked_button

    window = pygame.display.set_mode((1280,720))
    window.fill(black) # fill windows with black color
    create_play_and_menu_area_sections()
    draw_grid(window, size, num_rows) # grid will not change (no need to redraw)
    draw_menus()

    
    white = (255,255,255) # white color
    color_light = (170,170,170) # light shade of the button
    color_dark = (100,100,100) # dark shade of the button

    pygame.display.update() # update display

    exit = False
    while not exit: # main loop
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()

        QUIT_BUTTON = Button(image=None , pos=(1047, 600), 
                            text_input="QUIT", font=get_font(75), base_color="White", hovering_color="Red")

        OPTIONS_BACK = Button(image=None ,pos=(1047, 465), 
                            text_input="BACK", font=get_font(75), base_color="White", hovering_color="Green")
        OPTIONS_BACK.changeColor(OPTIONS_MOUSE_POS)
        OPTIONS_BACK.update(SCREEN)

        QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        QUIT_BUTTON.update(SCREEN)
        MENU_TEXT_1 = get_font(1).render("RUN", True, "#b68f40")
        MENU_TEXT_2 = get_font(1).render("BOT ", True, "#b68f40")
        MENU_TEXT_3 = get_font(1).render("goal ", True, "#b68f40")
        MENU_TEXT_4 = get_font(1).render("WALL", True, "#b68f40")
        MENU_RECT_1 = MENU_TEXT_1.get_rect(center=(1047, 294))
        MENU_RECT_2 = MENU_TEXT_2.get_rect(center=(1047, 177))
        MENU_RECT_3 = MENU_TEXT_3.get_rect(center=(1047, 60))
        MENU_RECT_4 = MENU_TEXT_4.get_rect(center=(1047, 390))            


        SCREEN.blit(MENU_TEXT_1, MENU_RECT_1)
        SCREEN.blit(MENU_TEXT_2, MENU_RECT_2)

        SCREEN.blit(MENU_TEXT_3, MENU_RECT_3)

        SCREEN.blit(MENU_TEXT_4, MENU_RECT_4)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if OPTIONS_BACK.checkForInput(OPTIONS_MOUSE_POS):
                            play()
                if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                            pygame.quit()
                            sys.exit()
            
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_presses = pygame.mouse.get_pressed()
                if mouse_presses[0]:
                    x, y = pygame.mouse.get_pos()
                    if menu_area_rect.collidepoint(pygame.mouse.get_pos()):
                        check_if_menu_button_pressed(x, y)
                        if clicked_button == 'run':
                            cliked_button = None
                           # print("run clicked")
                            if(len(goal_cells) > 0 and bot is not None):
                                #print("calling breadh firs search")
                                search_in_breadth_first_approach()
                    elif play_area_rect.collidepoint(pygame.mouse.get_pos()):
                        if clicked_button == 'wall':
                            create_wall_cell(x, y)
                        elif clicked_button == 'goal':
                            create_goal_cell(x, y)
                        elif clicked_button == 'bot':
                            create_bot(x, y)                          
        
        pygame.display.update()


def search_in_breadth_first_approach():
    global bot, goal_cells, path
    #print("x is: ", bot.x, "y is: ", bot.y)
    bot_current_cell_number = get_cell_number(bot.x, bot.y)
    
    # putting the intial bot position to open queue
    open_queue.append(bot_current_cell_number)

    processing = True
    find_path = False
    while processing: # main loop
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                exit()
    
        if len(open_queue) > 0 :
            current_node_cell = open_queue.pop(0) # remove and get first element of open_queue
            
            # check if this cell allready processed
            if(current_node_cell in close_queue):
                continue

            #print("current cell number is: ", current_node_cell)
            #print("goal cell/s is/are at: ", goal_cells)

            # cheking for goal
            if (current_node_cell in goal_cells):
                # add the goal cell to close queue too, this is helpfull to find the path
                close_queue.append(current_node_cell)
                found_goal_cell = current_node_cell # there can be many goal cell, this is the one found first.
                #print("goal found, now find the path from close_queue, close_queue is:", close_queue)
                processing = False
                find_path = True
                clock.tick(2) # stay some time here without closing the windoew
            else:
                # goal not found, continue processing
                child_node_cells = get_child_nodes(current_node_cell) # find possible child cells nodes to process
                close_queue.append(current_node_cell) # putting the processed node to closed queue

                # add child nodes to open queue only if they are already not in both open_queue and close_queue
                #print("childe nodes: ", child_node_cells)
                for child_node in child_node_cells:
                    if child_node not in open_queue and child_node not in close_queue:
                        open_queue.append(child_node) # add children to END OF THE OPEN QUEUE (BFS)
                #print("open queue: ", open_queue, "open_queue length: ", len(open_queue), "\n")

                paint_child_node_cells(child_node_cells)
            

        else:
            #print("no nodes to processe, open_queue length: ", len(open_queue), ", open queue: ", open_queue)
            #print("closed queue (processed nodes) : ", close_queue)
            return close_queue
    
    dead_end_nodes = []
    while find_path:
        path.append(close_queue[0]) # put the bot starting cell to path.

        for i in range(len(close_queue) -1):
            from_cell = path[-1]
            to_cell = close_queue[i+1]

            if to_cell in dead_end_nodes:
                continue

            #print("finding path, from_Cell: ", from_cell, ", try to_cell: ", to_cell)
            if verify_to_cell_is_navigatable_from_from_cell(from_cell, to_cell):
                path.append(to_cell)
        
        if path[-1] == found_goal_cell:
            find_path = False
        else:
            # a dead end has occured, start finding path avoiding this dead end cell
            dead_end_nodes.append(path[-1])
            path = [] # to start again

        
    paint_path(path)


def get_cell_number(x, y):
    #print("get_cell_number, x:", x, " y: ", y)
    cell_number = None
    for i in range(num_rows):  # columns
        for j in range(num_rows):  # rows
            if (x == j * line_width) and (y == i * line_width): # exact cell top left coordinates
                cell_number = (i * num_rows) + (j)
                #print("i :", i, ", j :", j)
                return cell_number
            
            elif (x < (j+1) * line_width) and (y < (i+1) * line_width): # soon as conditions met, that is the cell no. return
                # here j+1 and i+1 is because now need to consider the bottom right corner of the cell.
                cell_number = (i * num_rows) + (j)
                #print("i :", i, ", j :", j, ", cell number is: ", cell_number)
                return cell_number


def check_if_menu_button_pressed(x, y):
    global run_button, wall_button, bot_button, goal_button, clicked_button

    if run_button.collidepoint(pygame.mouse.get_pos()):
        clicked_button = 'run'
        #print("run clicccc")
    elif wall_button.collidepoint(pygame.mouse.get_pos()):
        clicked_button = 'wall'
    elif bot_button.collidepoint(pygame.mouse.get_pos()):
        clicked_button = 'bot'
    elif goal_button.collidepoint(pygame.mouse.get_pos()):
        clicked_button = 'goal'


def create_wall_cell(x, y):
    global window
    #print("create_wall_Cell, x:", x, " y: ", y)
    cell_no = get_cell_number(x, y) # get the cell number which mouse was clicked on
    cell_x, cell_y = get_top_left_cordinates_given_cell_number(cell_no)
    if not cell_no in wall_cells:
        wall_cells.append(cell_no)
        #print("cell number is: ", cell_no, ", type is: ", type(cell_no))
        pygame.draw.rect(window, white, pygame.Rect(cell_x, cell_y, line_width, line_width))

    else:
        wall_cells.remove(cell_no)
        pygame.draw.rect(window, black, pygame.Rect(cell_x, cell_y, line_width, line_width))
        draw_grid(window, size, num_rows) # adjust grid lines disapear, hence redraw


def create_goal_cell(x, y):
    global window
    cell_no = get_cell_number(x, y) # get the cell number which mouse was clicked on
    cell_x, cell_y = get_top_left_cordinates_given_cell_number(cell_no)
    if not cell_no in goal_cells:
        goal_cells.append(cell_no)
        #print("cell number is: ", cell_no, ", type is: ", type(cell_no))
        pygame.draw.rect(window, mat_green, pygame.Rect(cell_x, cell_y, line_width, line_width))
    else:
        goal_cells.remove(cell_no)
        pygame.draw.rect(window, black, pygame.Rect(cell_x, cell_y, line_width, line_width))
        draw_grid(window, size, num_rows) # adjust grid lines disapear, hence redraw


def create_bot(x, y):
    global window, bot_init_cell, bot
    if bot_init_cell is not None: # remove allready place bot
        cell_x, cell_y = get_top_left_cordinates_given_cell_number(bot_init_cell)
        pygame.draw.rect(window, black, pygame.Rect(cell_x, cell_y, line_width, line_width))   
    cell_no = get_cell_number(x, y) # get the cell number which mouse was clicked on
    cell_x, cell_y = get_top_left_cordinates_given_cell_number(cell_no)
    #print("cell number is: ", cell_no, ", type is: ", type(cell_no))   
    bot_init_cell = cell_no   
    bot = pygame.draw.rect(window, mat_blue, pygame.Rect(cell_x, cell_y, line_width, line_width))
    draw_grid(window, size, num_rows) # adjacent grid lines disapear, hence redraw


def get_child_nodes(cell_number):
    children = []
    up = get_up_cell(cell_number)
    if up is not None and up not in wall_cells:
        children.append(up)

    right = get_right_cell(cell_number)
    if right is not None and right not in wall_cells:
        children.append(right)

    down = get_down_cell(cell_number)
    if down is not None and down not in wall_cells:
        children.append(down)

    left = get_left_cell(cell_number)
    if left is not None and left not in wall_cells:
        children.append(left)
    
    return children

    

def get_up_cell(cell_number):
    cell_row_number = cell_number // num_rows # current row number of botpygame.image.load("assets/Quit Rect.png")
    if (cell_row_number - 1 < 0):  # above /up row number of bot
        return None
    else:
        return (cell_number - num_rows)


def get_right_cell(cell_number):
    cell_column_number = cell_number % num_rows # current column number of bot
    if (cell_column_number + 1 >= num_rows): 
        # current cell is at the right edge, so no rigth child / right cell available
        return None
    else:
        return (cell_number + 1) # else return next cell number


def get_down_cell(cell_number):
    cell_row_number = cell_number // num_rows # current row number of bot
    if (cell_row_number + 1 >= num_rows):  # down / next row number of bot
        return None
    else:
        return (cell_number + num_rows)


def get_left_cell(cell_number):
    cell_column_number = cell_number % num_rows # current column number of bot
    if (cell_column_number - 1 < 0): 
        # current cell is at the left edge, so no left child / right cell available
        return None
    else:
        return (cell_number - 1) # else return previous cell number


def paint_child_node_cells(child_node_cells):
    for child_node_cell in child_node_cells:
        if child_node_cell is not bot_init_cell and child_node_cell not in goal_cells:
            cell_x, cell_y = get_top_left_cordinates_given_cell_number(child_node_cell)
            pygame.draw.rect(window, child_node_blue, pygame.Rect(cell_x, cell_y, line_width, line_width))
            draw_grid(window, size, num_rows) # because adjust grid lines disappear, so redraw it again
            pygame.display.update()
            clock.tick(60)

def paint_path(path):
    #print("path is: ", path)
    for path_node_cell in path:
        if path_node_cell is not bot_init_cell and path_node_cell not in goal_cells:
            cell_x, cell_y = get_top_left_cordinates_given_cell_number(path_node_cell)
            pygame.draw.rect(window, path_cell_yellow, pygame.Rect(cell_x, cell_y, line_width, line_width))
            draw_grid(window, size, num_rows) # because adjust grid lines disappear, so redraw it again
            pygame.display.update()
            clock.tick(15)


def move_bot(cell_to_move):
    global window, bot, close_queue

    # clearing or painint another color for last cell
    if(len(close_queue) > 0):
        last_cell = close_queue[-1]
        last_x, last_y = get_top_left_cordinates_given_cell_number(last_cell)
        pygame.draw.rect(window, (50, 50, 50), pygame.Rect(last_x, last_y, line_width, line_width))

    x, y = get_top_left_cordinates_given_cell_number(cell_to_move)
    #print("moving to cell : ", cell_to_move, " of cordinates x: ", x, ", y: ", y, ",  line_width: ", line_width)
    bot = pygame.draw.rect(window, blue, pygame.Rect(x, y, line_width, line_width))
    #print("moved bot attributes: bot.x: ", bot.x, ", bot.y: ", bot.y)



def get_top_left_cordinates_given_cell_number(cell_to_move):
    cell_row_number = cell_to_move // num_rows # cell row number
    cell_column_number = cell_to_move % num_rows # cell column number

    y = cell_row_number * line_width
    x = cell_column_number * line_width
    return x, y


def verify_to_cell_is_navigatable_from_from_cell(from_cell, to_cell):
    if (to_cell in wall_cells): # if to_cell is a wall cell, return False
        return False

    if(from_cell + 1 == to_cell): # check to_cell is the right cell
        return True

    if(from_cell - 1 == to_cell): # check to_cell is the left cell
        return True
    
    if(from_cell - num_rows == to_cell): # check to_cell is the top / up cell
        return True

    if(from_cell + num_rows == to_cell): # check to_cell is the down / bottom cell
        return True
    
    return False # Else not navigatable, return False


def draw_grid(window, size, num_rows):
    x = 0
    y = 0

    for l in range(num_rows):
        x += line_width
        y += line_width

        pygame.draw.line(window, white, (x, 0), (x, size)) # drawing columns
        pygame.draw.line(window, white, (0, y), (size, y)) # drawing rows
def BFS_VIS():
    main()


In [28]:
def play():
    while True:
        PLAY_MOUSE_POS = pygame.mouse.get_pos()
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
        pygame.display.set_caption("Menu")

        BG = pygame.image.load("assets/Background.png")
        SCREEN.blit(BG, (0, 0))

        #PLAY_TEXT = get_font(45).render("This is the PLAY screen.", True, "White")
        #PLAY_RECT = PLAY_TEXT.get_rect(center=(640, 260))
        #SCREEN.blit(PLAY_TEXT, PLAY_RECT)


        #OPTIONS_TEXT = get_font(45).render("This is the OPTIONS screen.", True, "Black")
        #OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 100))
        #SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)

        ASTAR_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(640, 175), 
                            text_input="A*", font=get_font(30), base_color="#d7fcd4", hovering_color="Blue")

        
        Dijkstra_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(640, 360), 
                            text_input="DIJKSTRA ", font=get_font(75), base_color="#d7fcd4", hovering_color="Blue")
        
        Breadth_First_Search_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(640, 545), 
                            text_input="BFS", font=get_font(75), base_color="#d7fcd4", hovering_color="Blue")
        
        
        
        QUIT_BUTTON = Button(image=None, pos=(1200, 650), 
                            text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")
        
        OPTIONS_BACK = Button(image=None, pos=(50, 650), 
                            text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")
        


        
        ASTAR_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        ASTAR_BUTTON.update(SCREEN)
        

        
        Dijkstra_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        Dijkstra_BUTTON.update(SCREEN)
        
        Breadth_First_Search_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        Breadth_First_Search_BUTTON.update(SCREEN)
        
        
        OPTIONS_BACK.changeColor(OPTIONS_MOUSE_POS)
        OPTIONS_BACK.update(SCREEN)
        
        QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        QUIT_BUTTON.update(SCREEN)
        

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if OPTIONS_BACK.checkForInput(OPTIONS_MOUSE_POS):
                    main_menu()
                if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    pygame.quit()
                    sys.exit()
                if ASTAR_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    ASTAR_VIS()

                if Dijkstra_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    DIJKSTRA_VIS()
                if Breadth_First_Search_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    BFS_VIS()
                
                

        pygame.display.update()

In [29]:
def ASTAR():
    while True:
        PLAY_MOUSE_POS = pygame.mouse.get_pos()
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
        SCREEN.fill("gray")
        BG = pygame.image.load("assets/astar.jpg")
        SCREEN.blit(BG, (0, 0))
        OPTIONS_TEXT = get_font(45).render("This is the ASTAR screen.", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 100))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        PLAY_BACK = Button(image=None, pos=(640, 460), 
                                text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")

        PLAY_BACK.changeColor(PLAY_MOUSE_POS)
        PLAY_BACK.update(SCREEN)
        QUIT_BUTTON = Button(image=None, pos=(1047, 465), 
                            text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")
        
        QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        QUIT_BUTTON.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if PLAY_BACK.checkForInput(PLAY_MOUSE_POS):
                    options()
                if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    pygame.quit()
                    sys.exit()

        pygame.display.update()
    

In [30]:
def BIASTAR():
    while True:
        PLAY_MOUSE_POS = pygame.mouse.get_pos()
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
        BG = pygame.image.load("assets/astar.jpg")
        SCREEN.blit(BG, (0, 0))
        SCREEN.fill("gray")
        OPTIONS_TEXT = get_font(45).render("A* ALGORITHIM.", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 100))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        PLAY_BACK = Button(image=None, pos=(640, 460), 
                                text_input="BACK", font=get_font(75), base_color="White", hovering_color="Green")

        PLAY_BACK.changeColor(PLAY_MOUSE_POS)
        PLAY_BACK.update(SCREEN)
        QUIT_BUTTON = Button(image=None, pos=(1047, 465), 
                            text_input="QUIT", font=get_font(75), base_color="White", hovering_color="Red")
        
        QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        QUIT_BUTTON.update(SCREEN)

        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if PLAY_BACK.checkForInput(PLAY_MOUSE_POS):
                        options()
                    if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                        pygame.quit()
                        sys.exit()

        pygame.display.update()

In [31]:
def DIJKSTRA():
    while True:
        PLAY_MOUSE_POS = pygame.mouse.get_pos()
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
        BG = pygame.image.load("assets/diji.jpg")
        SCREEN.blit(BG, (0, 0))

        OPTIONS_TEXT = get_font(45).render("This is the DIJKSTRA screen.", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 100))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        PLAY_BACK = Button(image=None, pos=(640, 460), 
                                text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")

        PLAY_BACK.changeColor(PLAY_MOUSE_POS)
        PLAY_BACK.update(SCREEN)
        QUIT_BUTTON = Button(image=None, pos=(1047, 465), 
                            text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")
        
        QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        QUIT_BUTTON.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if PLAY_BACK.checkForInput(PLAY_MOUSE_POS):
                    options()
                if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    pygame.quit()
                    sys.exit()

        pygame.display.update()

In [32]:
def BFS():
    while True:

        pygame.display.set_caption("Menu")
        SCREEN.fill("gray")
        BG = pygame.image.load("assets/bfs.jpg")
        SCREEN.blit(BG, (0, 0))
        PLAY_MOUSE_POS = pygame.mouse.get_pos()
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()

        
        OPTIONS_TEXT = get_font(45).render("BREADTH FIRST SEARCH.", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 100))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        PLAY_BACK = Button(image=None, pos=(640, 465), 
                                text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")

        PLAY_BACK.changeColor(PLAY_MOUSE_POS)
        PLAY_BACK.update(SCREEN)
        QUIT_BUTTON = Button(image=None, pos=(1047, 465) , 
                            text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")
        
        QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        QUIT_BUTTON.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if PLAY_BACK.checkForInput(PLAY_MOUSE_POS):
                    options()
                if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    pygame.quit()
                    sys.exit()

        pygame.display.update()

In [33]:
def options():
    while True:
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()

        pygame.display.set_caption("Menu")

        BG = pygame.image.load("assets/Background.png")
        SCREEN.blit(BG, (0, 0))
        #OPTIONS_TEXT = get_font(45).render("This is the OPTIONS screen.", True, "Black")
        #OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 100))
        #SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)

        ASTAR_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(640, 175), 
                            text_input="A*", font=get_font(30), base_color="#d7fcd4", hovering_color="Blue")

        
        Dijkstra_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(640, 360), 
                            text_input="DIJKSTRA ", font=get_font(75), base_color="#d7fcd4", hovering_color="Blue")
        
        Breadth_First_Search_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(640, 545), 
                            text_input="BFS", font=get_font(75), base_color="#d7fcd4", hovering_color="Blue")
        
        
        
        QUIT_BUTTON = Button(image=None, pos=(1200, 650), 
                            text_input="QUIT", font=get_font(75), base_color="Black", hovering_color="Red")
        
        OPTIONS_BACK = Button(image=None, pos=(50, 650), 
                            text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")
        


        
        ASTAR_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        ASTAR_BUTTON.update(SCREEN)
        

        
        Dijkstra_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        Dijkstra_BUTTON.update(SCREEN)
        
        Breadth_First_Search_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        Breadth_First_Search_BUTTON.update(SCREEN)
        
        
        OPTIONS_BACK.changeColor(OPTIONS_MOUSE_POS)
        OPTIONS_BACK.update(SCREEN)
        
        QUIT_BUTTON.changeColor(OPTIONS_MOUSE_POS)
        QUIT_BUTTON.update(SCREEN)
        

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if OPTIONS_BACK.checkForInput(OPTIONS_MOUSE_POS):
                    main_menu()
                if QUIT_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    pygame.quit()
                    sys.exit()
                if ASTAR_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    ASTAR()

                if Dijkstra_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    DIJKSTRA()
                if Breadth_First_Search_BUTTON.checkForInput(OPTIONS_MOUSE_POS):
                    BFS()
                
                

        pygame.display.update()

In [34]:


pygame.init()

SCREEN = pygame.display.set_mode((1280, 720))
pygame.display.set_caption("Menu")

BG = pygame.image.load("assets/cs50image.jpg")

def get_font(size): # Returns Press-Start-2P in the desired size
    return pygame.font.SysFont('arialblack', 50)
def main_menu():
    while True:
        SCREEN.blit(BG, (0, 0))

        MENU_MOUSE_POS = pygame.mouse.get_pos()

        MENU_TEXT = get_font(100).render("MAIN MENU", True, "Black")
        MENU_RECT = MENU_TEXT.get_rect(center=(640, 100))

        PLAY_BUTTON = Button(image=None, pos=(150,365), 
                            text_input="VISUALIZATION", font=get_font(75), base_color="White", hovering_color="Black")
        OPTIONS_BUTTON = Button(image=None ,pos=(1150, 365), 
                            text_input="EXPLANTION", font=get_font(75), base_color="White", hovering_color="Black")
        QUIT_BUTTON = Button(image=None, pos=(640, 650), 
                            text_input="QUIT", font=get_font(75),base_color="White", hovering_color="Black")

        SCREEN.blit(MENU_TEXT, MENU_RECT)

        for button in [PLAY_BUTTON, OPTIONS_BUTTON, QUIT_BUTTON]:
            button.changeColor(MENU_MOUSE_POS)
            button.update(SCREEN)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if PLAY_BUTTON.checkForInput(MENU_MOUSE_POS):
                    play()
                if OPTIONS_BUTTON.checkForInput(MENU_MOUSE_POS):
                    options()
                if QUIT_BUTTON.checkForInput(MENU_MOUSE_POS):
                    pygame.quit()
                    sys.exit()

        pygame.display.update()

main_menu()

SystemExit: 